Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris) 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "Graphium"
data_name = "zinc12k"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='graphium', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [8]:
PATH = f"{gcp_root}/data/raw/ZINC12k.csv"
table = pd.read_csv(PATH)
table.columns

Index(['smiles', 'SA', 'logp', 'score'], dtype='object')

### Below we specify the meta information of data columns

In [10]:
# Additional meta-data on the column level
annotations = {
    "smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "SA": ColumnAnnotation(description="Synthetic accessibility score."),
    "logp": ColumnAnnotation(description="Log P, octanol-water partition coefficient."),
    "score": ColumnAnnotation(description="Score"),
}

### Define `Dataset` object

In [11]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [12]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="A subset (12K) of ZINC molecular graphs (250K) dataset.",
    source="https://arxiv.org/abs/2003.00982",
    annotations=annotations,
    tags=["Graph"],
    owner=owner,
    license="CC-BY-4.0",
    user_attributes={"year": "2022"},
    readme=load_readme("org-Graphium/zinc12/zinc12_readme.md"),
)

In [13]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

2024-07-16 23:05:28.217 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


'gs://polaris-public/polaris-recipes/org-Graphium/zinc12k/datasets/zinc12k-v1/dataset.json'

In [14]:
from polaris.hub.client import PolarisHubClient

client = PolarisHubClient()
client.login()

client.upload_dataset(dataset=dataset, access="private", owner=owner)

2024-07-16 23:05:37.048 | INFO     | polaris.hub.client:login:285 - You are already logged in to the Polaris Hub as  (lu@valencediscovery.com). Set `overwrite=True` to force re-authentication.
2024-07-16 23:05:39.938 | SUCCESS  | polaris.hub.client:upload_dataset:631 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/graphium/zinc12k-v1


{'id': 'lyuMH2u2azzmsvdwtrMu3',
 'createdAt': '2024-07-17T03:05:37.875Z',
 'deletedAt': None,
 'name': 'zinc12k-v1',
 'slug': 'zinc12k-v1',
 'description': 'A subset (12K) of ZINC molecular graphs (250K) dataset.',
 'tags': ['Graph'],
 'userAttributes': {'year': '2022'},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'readme': '## Background\n\n\n## Assay information\n\n\n## Description of readout:\n\n\n## Data resource\n\n',
 'ownerId': 'zMTB7lQiiukqEmLQF7EjT',
 'creatorId': 'NKnaHGybLqwSHcaMEHqfF',
 'state': 'upload_pending',
 'source': 'https://arxiv.org/abs/2003.00982',
 'curationReference': None,
 'nRows': 12000,
 'nColumns': 4,
 'license': 'CC-BY-4.0',
 'md5Sum': '804d595f51d3841f329fdb4fec03cc60',
 'annotations': {'SA': {'dtype': 'float64',
   'modality': 'UNKNOWN',
   'isPointer': False,
   'description': 'Synthetic accessibility score.',
   'userAttributes': {}},
  'logp': {'dtype': 'float64',
   'modality': 'UNKNOWN',
   'isPointer': False,
   'descri